# User Songs!

A brief diversion into how exactly the user songs are represented.

In [1]:
import mido

In [5]:
from commons.messages import controlstate

I've recorded one track (track 3) on one user song (user song 2).

In [6]:
!python extractor.py documents/data/user_song_tests/1track.syx -S 2

               Recorded   Duration       Size
 User Song 2       True         41       2112
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True         41       1999
     Track 4      False          0          0
     Track 5      False          0          0



What's in the MIDI?

In [8]:
!python extractor.py documents/data/user_song_tests/1track.syx -s 2 -n documents/data/user_song_tests/1track_{}.mid

In [9]:
mf = mido.MidiFile('documents/data/user_song_tests/1track_2.mid')

In [10]:
mf

<midi file 'documents/data/user_song_tests/1track_2.mid' type 1, 2 tracks, 499 messages>

In [13]:
mf.tracks

[<midi track '' 10 messages>, <midi track '' 489 messages>]

We have the time track, which we put first, and then all the other tracks.

In [17]:
mf.print_tracks()

=== Track 0
<meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message set_tempo tempo=400000 time=0>
<message sysex data=(126,127,9,1) time=0>
<message sysex data=(67,21,76,2,1,0,1,0) time=0>
<message sysex data=(67,21,76,2,1,32,66,17) time=0>
<meta message text text=' YAMAHA ' time=0>
<meta message sequencer_specific data=(67, 118, 26, 4, 0, 0) time=0>
<meta message sequencer_specific data=(67, 118, 26, 2, 100) time=0>
<meta message sequencer_specific data=(67, 118, 26, 1, 0) time=0>
<meta message end_of_track time=0>
=== Track 1
<message sysex data=(67,18,76,2,1,0,1,0) time=0>
<message sysex data=(67,18,76,2,1,32,66,17) time=0>
<meta message text text=' YAMAHA ' time=0>
<message control_change channel=2 control=0 value=0 time=0>
<message control_change channel=2 control=32 value=112 time=0>
<message program_change channel=2 program=0 time=0>
<message control_change channel=12 control=0 value=0 time=0>
<message contr

In [18]:
mido.tempo2bpm(400000)

150.0

In [36]:
def hexspace(x):
    return " ".join(format(b, "02X") for b in x)

def trackprint(track):
    s = controlstate.MidiControlState()
    t = 0
    for m in track:
        t += m.time
        if m.is_meta:
            if m.type == 'sequencer_specific':
                print(t, 'Sequencer Specific', hexspace(m.data))
            else:
                print(t, m)
        else:
            w = s.feed(m)
            if w:
                print(t, w)
            else:
                print(t, m)


In [37]:
trackprint(mf.tracks[0])

0 <meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=400000 time=0>
0 GM System ON
0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In the time track, we have the time signature and tempo, along with reverb and chorus type. There's a meta message for YAMAHA text, and some sequencer specific messages; all this happens at time 0.

In [38]:
trackprint(mf.tracks[1])

0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 2 Bank MSB 0
0 2 Bank LSB 112
0 2 Program Change [0,112,0] 003 Grand Piano (PIANO)
0 C Bank MSB 0
0 C Bank LSB 112
0 C Program Change [0,112,85] 103 Voice Lead (SYNTH LEAD)
0 2 RPN MSB 0
0 2 RPN LSB 0
0 2 Data Entry MSB: Pitch Bend Range 2
0 2 RPN LSB 127
0 2 RPN MSB 127
0 C RPN MSB 0
0 C RPN LSB 0
0 C Data Entry MSB: Pitch Bend Range 2
0 C RPN LSB 127
0 C RPN MSB 127
0 2 Voice Reverb Level 20
0 2 Voice Chorus Level 41
0 2 [Control 94] 0
0 2 Voice Volume 100
0 2 Voice Pan 54
0 2 Expression 127
0 2 Release Time 0
0 2 Pedal Sustain OFF
0 C Voice Reverb Level 60
0 C Voice Chorus Level 24
0 C [Control 94] 0
0 C Voice Volume 100
0 C Voice Pan 74
0 C Expression 127
0 C Release Time 0
0 C Pedal Sustain OFF
0 polytouch channel=2 note=0 value=63 time=0
0 2 Pitch Bend 0
0 C Pitch Bend 0
0 2 Note 036(C1) 77
0 C Note 036(C1) 77
281 2 Note 036(C1) 0
281 C Note 036(C1) 0
285 2 Note 043(G1) 82
285 C Note 0

I recorded Track 3, with main voice 003 and dual voice 103. It seems the main voice is on Channel 0x2 and the dual voice on channel 0xC (so, Channels 3 and 13).

At the beginning, the controls are set for reverb and chorus type, pitch bend, reverb and chorus levels, voice volume, pan, expression, release time (panel sustain), as wekk as Control 94, which according to the MIDI spec table is Effects 4 (Celeste) depth, which is set to zero. The DGX-505 does not apparently recognise it or emit it, but it's something that's recorded (for compatibility?) Same with the polyphonic aftertouch.



Now, what's sent over the MIDI on Song Out playback?

In [40]:
!python slurp.py -gp DGX > documents/data/user_song_tests/1track_2.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [45]:
with open('documents/data/user_song_tests/1track_2.txt') as infile:
    played = list(mido_util.readin_strings(infile))

In [44]:
from commons import mido_util

In [68]:
def mprint(seq, tpb=None, bpm=None):
    s = controlstate.MidiControlState()
    rt = 0
    if tpb:
        tempo = mido.bpm2tempo(bpm)
        
    for m in seq:
        if m.type != 'clock':
            if m.type == 'start':
                rt = m.time
            t = m.time - rt
            w = s.feed(m)
            
            if tpb:
                t = mido.second2tick(t, tpb, tempo)
            
            if w:
                print(format(t, '.2f'), w)
            else:
                print(format(t, '.2f'), m)


In [69]:
mprint(played, 96, 150)

0.00 start time=2.2532950180002445
2.66 5 All Sound OFF
2.88 5 Note 024(C0) 77
283.93 5 Note 024(C0) 0
287.76 5 Note 031(G0) 82
565.68 5 Note 026(D0) 65
568.79 5 Note 031(G0) 0
842.62 5 Note 026(D0) 0
846.70 5 Note 029(F0) 68
1145.74 5 Note 029(F0) 0
1147.66 5 Note 019(G-1) 70
1238.61 5 Note 059(B2) 65
1329.82 5 Note 059(B2) 0
1331.73 5 Note 062(D3) 72
1420.53 5 Note 029(F0) 56
1421.01 5 Note 062(D3) 0
1422.69 5 Note 069(A3) 71
1438.77 5 Note 019(G-1) 0
1508.85 5 Note 067(G3) 63
1518.69 5 Note 069(A3) 0
1596.71 5 Note 065(F3) 75
1609.66 5 Note 067(G3) 0
1687.64 5 Note 024(C0) 55
1692.69 5 Note 064(E3) 79
1698.68 5 Note 029(F0) 0
1701.56 5 Note 065(F3) 0
1920.68 5 Note 064(E3) 0
2006.84 5 Note 024(C0) 0
2291.71 5 Note 029(F0) 65
2384.60 5 Note 065(F3) 65
2473.63 5 Note 065(F3) 0
2478.66 5 Note 069(A3) 70
2546.82 5 Note 069(A3) 0
2576.82 5 Note 081(A4) 61
2666.82 5 Note 081(A4) 0
2667.78 5 Note 079(G4) 66
2752.74 5 Note 029(F0) 0
2757.78 5 Note 077(F4) 63
2764.74 5 Note 079(G4) 0
2834.82

In [50]:
!python slurp.py -ngp DGX > documents/data/user_song_tests/1track_ini.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [51]:
with open('documents/data/user_song_tests/1track_ini.txt') as infile:
    ini1 = list(mido_util.readin_strings(infile))

In [70]:
mprint(ini1, 96, 150)

574.68 GM System ON
622.90 XG System ON
670.91 MIDI Master Tuning 0
671.85 Reverb Type 01(Hall1)
672.57 Chorus Type 1(Chorus1)
694.41 3 Bank MSB 0
694.43 3 Bank LSB 0
694.64 3 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
694.88 3 RPN MSB 0
695.12 3 RPN LSB 0
695.14 3 Data Entry MSB: Pitch Bend Range 2
695.36 3 RPN LSB 127
695.38 3 RPN MSB 127
695.60 3 Voice Reverb Level 40
695.84 3 Voice Chorus Level 0
696.02 3 Voice Volume 100
696.03 3 Voice Pan 64
696.32 3 Pedal Sustain OFF
696.34 3 Modulation Wheel 0
696.56 3 Expression 127
696.80 3 Harmonic Content 0
696.82 3 Release Time 0
697.04 3 Attack Time 0
697.28 3 Brightness 0
697.52 3 Pitch Bend 0
697.76 4 Bank MSB 0
697.78 4 Bank LSB 0
698.00 4 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
698.25 4 RPN MSB 0
698.26 4 RPN LSB 0
698.48 4 Data Entry MSB: Pitch Bend Range 2
698.72 4 RPN LSB 127
698.74 4 RPN MSB 127
698.96 4 Voice Reverb Level 40
699.20 4 Voice Chorus Level 0
699.22 4 Voice Volume 100
699.44 4 Voice Pan 64
699.68 4 Pe

Interesting things to note:

The SONG midi output only includes the Main voice, on channel `5`. The Dual voice is not present (I suspect this is because the is no more room. `0` through `2` are keyboard voice, then `3` through `7` are for song tracks, and `8` through `F` are for style and accompaniment.

The track was recorded with MAIN OCTAVE set to -1, DUAL OCTAVE set to 0. This is reflected in the output from the SONG out, which are shifted one octave down, but not for the data stored in the MID file... This data must be *somewhere*... are they repurposing the polyphonic aftertouch??

If this is indeed the case, then for proper export we must actually parse the messages to correct for this!



In [71]:
!python collect.py -g DGX > documents/data/user_song_tests/2track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [72]:
!python extractor.py documents/data/user_song_tests/2track.syx -S 2 -s 2 -n documents/data/user_song_tests/2track_{}.mid

               Recorded   Duration       Size
 User Song 2       True         35       3805
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True         31       1999
     Track 4      False          0          0
     Track 5       True         35       1693



In [73]:
!python slurp.py -ngp DGX > documents/data/user_song_tests/2track_ini.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [75]:
with open('documents/data/user_song_tests/2track_ini.txt') as infile:
    ini2 = list(mido_util.readin_strings(infile))
mprint(ini2)

5.37 GM System ON
5.57 XG System ON
5.77 MIDI Master Tuning 0
5.77 Reverb Type 01(Hall1)
5.78 Chorus Type 1(Chorus1)
5.87 3 Bank MSB 0
5.87 3 Bank LSB 0
5.87 3 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
5.87 3 RPN MSB 0
5.87 3 RPN LSB 0
5.87 3 Data Entry MSB: Pitch Bend Range 2
5.87 3 RPN LSB 127
5.87 3 RPN MSB 127
5.87 3 Voice Reverb Level 40
5.87 3 Voice Chorus Level 0
5.87 3 Voice Volume 100
5.88 3 Voice Pan 64
5.88 3 Pedal Sustain OFF
5.88 3 Modulation Wheel 0
5.88 3 Expression 127
5.88 3 Harmonic Content 0
5.88 3 Release Time 0
5.88 3 Attack Time 0
5.88 3 Brightness 0
5.88 3 Pitch Bend 0
5.88 4 Bank MSB 0
5.88 4 Bank LSB 0
5.88 4 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
5.88 4 RPN MSB 0
5.88 4 RPN LSB 0
5.89 4 Data Entry MSB: Pitch Bend Range 2
5.89 4 RPN LSB 127
5.89 4 RPN MSB 127
5.89 4 Voice Reverb Level 40
5.89 4 Voice Chorus Level 0
5.89 4 Voice Volume 100
5.89 4 Voice Pan 64
5.89 4 Pedal Sustain OFF
5.89 4 Modulation Wheel 0
5.89 4 Expression 127
5.89 4 Harmo

Recorded another track, this time with a different tempo.
The Main voice was voice 205 with octave -2, the dual voice was voice 005 with octave +2.

In [76]:
mf2 = mido.MidiFile('documents/data/user_song_tests/2track_2.mid')

mf2

In [79]:
trackprint(mf2.tracks[0])

0 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=500000 time=0>
0 GM System ON
0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


The tempo is overwritten,  and Whoops, I apparently overwrote the time signature as well.

In [81]:
mf2.tracks[1] == mf.tracks[1]

True

In [82]:
trackprint(mf2.tracks[2])

0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 4 Bank MSB 0
0 4 Bank LSB 0
0 4 Program Change [0,0,22] 205 Harmonica (XG ORGAN)
0 E Bank MSB 0
0 E Bank LSB 112
0 E Program Change [0,112,3] 005 Honky-tonk Piano (PIANO)
0 4 RPN MSB 0
0 4 RPN LSB 0
0 4 Data Entry MSB: Pitch Bend Range 2
0 4 RPN LSB 127
0 4 RPN MSB 127
0 E RPN MSB 0
0 E RPN LSB 0
0 E Data Entry MSB: Pitch Bend Range 2
0 E RPN LSB 127
0 E RPN MSB 127
0 4 Voice Reverb Level 28
0 4 Voice Chorus Level 0
0 4 [Control 94] 0
0 4 Voice Volume 104
0 4 Voice Pan 64
0 4 Expression 127
0 4 Release Time 0
0 4 Pedal Sustain OFF
0 E Voice Reverb Level 32
0 E Voice Chorus Level 0
0 E [Control 94] 0
0 E Voice Volume 65
0 E Voice Pan 64
0 E Expression 127
0 E Release Time 0
0 E Pedal Sustain OFF
0 polytouch channel=4 note=0 value=62 time=0
0 4 Pitch Bend 0
0 E Pitch Bend 0
0 4 Note 060(C3) 50
0 E Note 084(C5) 50
0 4 Note 072(C4) 51
0 E Note 096(C6) 51
7 4 Pedal Sustain ON
7 E Pedal Sustain ON

The Main voice was recorded onto channel `4` (5), and dual voice onto channel `E` (15).

The Dual notes are recorded with +2 octave on them, but the main voice is not. Instead, it's got a polytouch message with value 62. Hmmm.

What happens if we try to change the voice while recording?

We can't use the function menus, but what about scrolling the voice.
Do the defaults get set?

What about using values saved in the bank registers?

What happens if we change the reverb and chorus type?

What about transpose?


In [91]:
!python extractor.py documents/data/user_song_tests/2track.syx -R 2,2 3,2

Bank 2, Button 2:
       Style number: 050
      Accompaniment: OFF
           Main A/B: Main B
              Tempo: 082
  Main Voice number: 078
  Dual Voice number: 076
 Split Voice number: 046
            Harmony:  ON
               Dual:  ON
              Split: OFF
       Style Volume: 100
          Transpose:  00
   Pitch Bend Range:  02
        Split Point: 054(F#2)
          M. Volume: 103
          M. Octave:   0
             M. Pan: 064
    M. Reverb Level: 040
    M. Chorus Level: 000
          D. Volume: 066
          D. Octave:  -1
             D. Pan: 064
    D. Reverb Level: 040
    D. Chorus Level: 020
          S. Volume: 100
          S. Octave:   0
             S. Pan: 064
    S. Reverb Level: 008
    S. Chorus Level: 000
        Reverb Type: 03(Hall3)
        Chorus Type: 1(Chorus1)
            Sustain: OFF
       Harmony Type: 01(Duet)
     Harmony Volume: 080

Bank 3, Button 2:
       Style number: 092
      Accompaniment:  ON
           Main A/B: Main A
         

|Main Voice                          |M Volume|M Octave|M Reverb Lv|M Chorus Lv|Dual Voice                     |D Volume|D Octave|D Reverb Lv|D Chorus Lv|
|:-----------------------------------|-------:|-------:|----------:|----------:|:------------------------------|-------:|-------:|----------:|----------:|
|085 Trombone Section                |     110|      -1|         34|          0|088 Brass Section              |      90|      -1|         32|          0|
|086 French Horn                     |      80|       0|         40|          0|084 Trombone                   |      70|       0|         36|          0|
|087 Tuba                            |     127|      -2|         18|          0|045 Acoustic Bass              |      84|      -2|          0|          0|
|119 Music Box                       |     100|      +1|         20|          0|108 SweetHeaven                |      60|       0|         42|          0|
|075 Baritone Sax                    |     112|      -2|         28|          0|088 Brass Section              |     100|      -1|         24|          0|
|025 16'+2' Organ                    |      70|      -1|         28|         34|026 16'+4' Organ               |      30|      +1|         28|          0|


Let's record track 4, starting with the chorus and reverb types set to something different. Then, cycle through voices 085, 086, 087, 119, 075, 025, with dual ON.



In [93]:
!python collect.py -g DGX > documents/data/user_song_tests/3track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [94]:
!python extractor.py documents/data/user_song_tests/3track.syx -S 2 -s 2 -n documents/data/user_song_tests/3track_{}.mid

               Recorded   Duration       Size
 User Song 2       True        143       5578
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True        127       1999
     Track 4       True        133       1773
     Track 5       True        143       1693



In [95]:
mf3 = mido.MidiFile('documents/data/user_song_tests/3track_2.mid')

In [96]:
mf3.tracks[1] == mf2.tracks[1]

True

In [97]:
mf3.tracks[3] == mf2.tracks[2]

True

In [99]:
trackprint(mf3.tracks[0])

0 <meta message time_signature numerator=1 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=454528 time=0>
0 GM System ON
0 Reverb Type 08(Plate1)
0 Chorus Type 4(Flanger2)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


The reverb and chorus type got overwritten. Also, I tried setting the time signature to 0, and it's at 1/4, which makes sense.

In [103]:
trackprint(mf3.tracks[2])

0 Reverb Type 08(Plate1)
0 Chorus Type 4(Flanger2)
0 <meta message text text=' YAMAHA ' time=0>
0 3 Bank MSB 0
0 3 Bank LSB 112
0 3 Program Change [0,112,62] 091 Synth Brass (BRASS)
0 D Bank MSB 0
0 D Bank LSB 112
0 D Program Change [0,112,79] 100 Ocarina (FLUTE)
0 3 RPN MSB 0
0 3 RPN LSB 0
0 3 Data Entry MSB: Pitch Bend Range 2
0 3 RPN LSB 127
0 3 RPN MSB 127
0 D RPN MSB 0
0 D RPN LSB 0
0 D Data Entry MSB: Pitch Bend Range 2
0 D RPN LSB 127
0 D RPN MSB 127
0 3 Voice Reverb Level 32
0 3 Voice Chorus Level 54
0 3 [Control 94] 0
0 3 Voice Volume 100
0 3 Voice Pan 43
0 3 Expression 127
0 3 Release Time 0
0 3 Pedal Sustain OFF
0 D Voice Reverb Level 87
0 D Voice Chorus Level 80
0 D [Control 94] 0
0 D Voice Volume 40
0 D Voice Pan 86
0 D Expression 127
0 D Release Time 0
0 D Pedal Sustain OFF
0 polytouch channel=3 note=0 value=64 time=0
0 3 Pitch Bend 0
0 D Pitch Bend 0
0 3 Note 053(F2) 39
0 D Note 041(F1) 39
0 3 Note 072(C4) 45
0 D Note 060(C3) 45
0 3 Note 077(F4) 42
0 D Note 065(F3) 42
0 

Okay, it definitely looks like the default settings get set. The Dual octave affects the dual notes directly, while the Main octave is signalled by a polytouch message, with offset from 64. The program changes etc. are recorded for both main and dual, even if dual is not activated. Transpose is completely ignored.

Question: Does the polytouch message work for songs in the flash memory?

What happens if we send a GM_ON or set the reverb in the middle? Does that reset the reverb and chorus types? How about regular reverb chorus messages?

Also, it seems that the durations are indeed in measures, and affected by the time signature on the time track.

Does setting LOCAL have any effect?

In [104]:
o = mido.open_output('DGX-505 MIDI 1')

In [107]:
from commons.messages import controls
import time

In [112]:
def test():
    o.send(controls.reverb_type(00, 00))
    time.sleep(2)
    o.send(controls.chorus_type(00, 00))
    time.sleep(4)
    o.send(controls.gm_on())
    time.sleep(2)
    o.send(controls.local(False))
    time.sleep(5)
    o.send(controls.local(True))
    time.sleep(3)
    o.send(controls.master_tune_val(100))
    time.sleep(3)
    o.send(controls.xg_reset())



In [113]:
test()

In [114]:
!python collect.py -g DGX > documents/data/user_song_tests/4track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [115]:
!python extractor.py documents/data/user_song_tests/4track.syx -S 2 -s 2 -n documents/data/user_song_tests/4track_{}.mid

               Recorded   Duration       Size
 User Song 2       True          9       7524
     Track A      False          0         99
     Track 1      False          0          0
     Track 2       True          6       1946
     Track 3       True          8       1999
     Track 4       True          8       1773
     Track 5       True          9       1693



In [116]:
o.close()

In [118]:
mf4 = mido.MidiFile('documents/data/user_song_tests/4track_2.mid')

In [120]:
mf4.tracks[2:] == mf3.tracks[1:]

True

In [121]:
trackprint(mf4.tracks[0])

0 <meta message time_signature numerator=15 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=600000 time=0>
0 GM System ON
0 Reverb Type 09(Plate2)
0 Chorus Type 3(Flanger1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In [122]:
trackprint(mf4.tracks[1])

0 Reverb Type 09(Plate2)
0 Chorus Type 3(Flanger1)
0 <meta message text text=' YAMAHA ' time=0>
0 1 Bank MSB 0
0 1 Bank LSB 113
0 1 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 B Bank MSB 0
0 B Bank LSB 112
0 B Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 1 RPN MSB 0
0 1 RPN LSB 0
0 1 Data Entry MSB: Pitch Bend Range 5
0 1 RPN LSB 127
0 1 RPN MSB 127
0 B RPN MSB 0
0 B RPN LSB 0
0 B Data Entry MSB: Pitch Bend Range 5
0 B RPN LSB 127
0 B RPN MSB 127
0 1 Voice Reverb Level 20
0 1 Voice Chorus Level 0
0 1 [Control 94] 0
0 1 Voice Volume 110
0 1 Voice Pan 64
0 1 Expression 127
0 1 Release Time 46
0 1 Pedal Sustain ON
0 B Voice Reverb Level 50
0 B Voice Chorus Level 0
0 B [Control 94] 0
0 B Voice Volume 36
0 B Voice Pan 64
0 B Expression 127
0 B Release Time 46
0 B Pedal Sustain ON
0 polytouch channel=1 note=0 value=66 time=0
0 1 Pitch Bend 8190
0 B Pitch Bend 8190
0 1 Note 048(C2) 67
63 1 Pitch Bend 7540
63 B Pitch Bend 7540
65 1 Pitch Bend 7020
65 B Pitch Bend 7020

The reverb and chorus type don't get changed while the thing is recording. LOCAL and so on do not seem to have an effect on what is recorded

In [123]:
with open('documents/data/user_song_tests/4track_rec.txt') as infile:
    recd = list(mido_util.readin_strings(infile))

In [128]:
mprint(recd, 96, 100)

2729.09 0 Pitch Bend 260
2729.23 1 Pitch Bend 260
2729.39 2 Pitch Bend 260
2729.88 0 Pitch Bend 780
2730.03 1 Pitch Bend 780
2730.19 2 Pitch Bend 780
2731.18 0 Pitch Bend 1300
2731.20 1 Pitch Bend 1300
2731.33 2 Pitch Bend 1300
2732.11 0 Pitch Bend 1820
2732.14 1 Pitch Bend 1820
2732.15 2 Pitch Bend 1820
2732.75 0 Pitch Bend 2470
2732.91 1 Pitch Bend 2470
2732.92 2 Pitch Bend 2470
2733.57 0 Pitch Bend 2990
2733.74 1 Pitch Bend 2990
2733.75 2 Pitch Bend 2990
2734.35 0 Pitch Bend 3250
2734.51 1 Pitch Bend 3250
2734.52 2 Pitch Bend 3250
2735.16 0 Pitch Bend 3770
2735.31 1 Pitch Bend 3770
2735.47 2 Pitch Bend 3770
2735.98 0 Pitch Bend 4290
2736.13 1 Pitch Bend 4290
2736.29 2 Pitch Bend 4290
2736.77 0 Pitch Bend 4550
2736.92 1 Pitch Bend 4550
2737.13 2 Pitch Bend 4550
2737.56 0 Pitch Bend 5070
2737.71 1 Pitch Bend 5070
2737.87 2 Pitch Bend 5070
2738.19 0 Pitch Bend 5330
2738.35 1 Pitch Bend 5330
2738.52 2 Pitch Bend 5330
2739.01 0 Pitch Bend 5850
2739.15 1 Pitch Bend 5850
2739.31 2 Pitch Be

4940.22 0 Note 098(D6) 0
4945.65 0 Note 091(G5) 0
4945.97 0 Note 089(F5) 0
4964.54 5 Note 067(G3) 56
4971.61 5 Note 072(C4) 0
4996.71 0 Note 091(G5) 38
4996.85 0 Note 100(E6) 40
4997.01 0 Note 089(F5) 53
5015.57 0 Note 101(F6) 0
5017.82 0 Note 086(D5) 0
5022.13 0 Note 089(F5) 0
5024.05 0 Note 091(G5) 0
5069.65 5 Note 060(C3) 71
5071.59 5 Note 067(G3) 0
5073.49 0 Note 079(G4) 39
5075.67 0 Note 095(B5) 39
5086.29 0 Note 100(E6) 0
5136.23 0 Note 098(D6) 43
5138.13 0 Note 089(F5) 37
5140.12 0 Note 086(D5) 37
5143.41 0 Note 083(B4) 30
5153.65 0 Note 095(B5) 0
5158.61 7 Note 053(F2) 55
5158.62 7 Note 041(F1) 52
5158.93 6 Note 060(C3) 53
5161.65 7 Pedal Sustain OFF
5162.29 0 Note 086(D5) 0
5163.41 0 Note 089(F5) 0
5173.65 5 Note 029(F0) 61
5176.55 5 Note 069(A3) 79
5177.01 5 Note 024(C0) 0
5177.49 5 Note 060(C3) 0
5192.55 7 Pedal Sustain ON
5202.45 0 Note 083(B4) 0
5221.97 0 Note 100(E6) 59
5222.45 0 Note 089(F5) 38
5223.09 0 Note 083(B4) 27
5225.18 0 Note 086(D5) 43
5231.89 0 Note 098(D6) 0


8871.17 5 Bank MSB 0
8871.18 5 Bank LSB 112
8871.33 5 Program Change [0,112,0] 003 Grand Piano (PIANO)
8871.49 5 RPN MSB 0
8871.65 5 RPN LSB 0
8871.81 5 Data Entry MSB: Pitch Bend Range 2
8871.82 5 RPN LSB 127
8871.97 5 RPN MSB 127
8872.13 5 Voice Reverb Level 20
8872.29 5 Voice Chorus Level 41
8872.45 5 Voice Pan 54
8872.46 5 Release Time 0
8873.25 5 Pitch Bend 0
8873.73 4 Bank MSB 0
8873.74 4 Bank LSB 113
8873.89 4 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
8874.05 4 RPN MSB 0
8874.21 4 RPN LSB 0
8874.38 4 Data Entry MSB: Pitch Bend Range 5
8874.39 4 RPN LSB 127
8874.60 4 RPN MSB 127
8874.70 4 Voice Reverb Level 20
8874.86 4 Voice Chorus Level 0
8875.01 4 Voice Pan 64
8875.03 4 Release Time 46
8875.65 4 Pitch Bend 8190
8877.89 Reverb Type 09(Plate2)
8878.37 Chorus Type 3(Flanger1)
